# MyScale

This notebook shows how to use functionality related to the MyScale vector database.

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import MyScale
from langchain.document_loaders import TextLoader

## Setting up envrionments

There are two ways to set up parameters for myscale index.

1. Environment Variables

    Before you run the app, please set the environment variable with `export`:

    `export myscale_host='<your-endpoints-url>' myscale_port=<your-endpoints-port> myscale_username=<your-username> ...`

    Every attributes under `MyScaleSettings` can be set with prefix `myscale` and is case insensitive.

2. Create `MyScaleSettings` object with parameters


    ```python
    from langchain.vectorstores import MyScale, MyScaleSettings
    config = MyScaleSetting(host="localhost", port=8123, ...)
    index = MyScale(embedding_function, config)
    index.add_documents(...)
    ```

In [3]:
from langchain.document_loaders import TextLoader
loader = TextLoader('../../../state_of_the_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [4]:
docsearch = MyScale.from_documents(docs, embeddings.embed_query)
                                   

query = "What did the president say about Ketanji Brown Jackson"
docs = docsearch.similarity_search(query)

Inserting data...: 100%|██████████| 42/42 [00:22<00:00,  1.90it/s]


In [5]:
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


## Show data schema

In [6]:
print(str(docsearch))

default.vector_table @ msc-3dfcbf0c.ap-southeast-1.aws.dev.myscale.cloud:8443

username: mpsk

Table Schema:
---------------------------------------------------
|id                      |String                  |
|text                    |String                  |
|vector                  |Array(Float32)          |
|metadata                |Object('json')          |
---------------------------------------------------



## Filtering

You can have direct access to myscale SQL where statement. You can write `WHERE` clause following standard SQL.

**NOTE**: Please be aware of SQL injection, this interface must not be directly called by end-user.

If you custimized your `column_map` under your setting, you search with filter like this:
```python
from langchain.vectorstores import MyScale, MyScaleSettings
from langchain.document_loaders import TextLoader

loader = TextLoader('../../../state_of_the_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

docsearch = MyScale.from_documents(docs, embeddings.embed_query)
docsearch.similarity_search_with_relevance_scores('What did the president say about Ketanji Brown Jackson?', 
    where_str=f"{docsearch.config.column_map['metadata']}.attribute>10 and \
        {docsearch.config.column_map['metadata']}.another_attribute='another string'")
```

## On deleting your data

In [7]:
docsearch.client.command(f'DROP TABLE {docsearch.config.database}.{docsearch.config.table}')

''